In [5]:
import os
import json
import requests
import pandas as pd
import utils.utils as utils
from tqdm import tqdm
from pymongo import MongoClient


dataset = {
    "Round1_T2D": {
        "tables": "./data/Dataset/Round1_T2D/tables",
        "cea":"./data/Dataset/Round1_T2D/gt/CEA_Round1_gt.csv",
        "cpa":"./data/Dataset/Round1_T2D/gt/CPA_Round1_gt.csv",
        "cta": "./data/Dataset/Round1_T2D/gt/CTA_Round1_gt.csv"
    },
    "Round3": {
        "tables": "./data/Dataset/Round3_2019/tables/",
        "cea":"./data/Dataset/Round3_2019/gt/CEA_Round3_gt.csv",
        "cpa":"./data/Dataset/Round3_2019/gt/CPA_Round3_gt.csv",
        "cta": "./data/Dataset/Round3_2019/gt/CTA_Round3_gt.csv"
    },
    "Round4": {
        "tables": "./data/Dataset/Round4_2020/tables/",
        "cea":"./data/Dataset/Round4_2020/gt/cea.csv",
        "cpa":"./data/Dataset/Round4_2020/gt/cpa.csv",
        "cta": "./data/Dataset/Round4_2020/gt/cta.csv"
    },
    "2T-2020": {
        "tables":"./data/Dataset/2T_Round4/tables/", 
        "cea":"./data/Dataset/2T_Round4/gt/cea.csv",
        "cpa": None,
        "cta": "./data/Dataset/2T_Round4/gt/cta.csv"
    },
    "HardTableR2-2021": {
        "tables":"./data/Dataset/HardTablesR2/tables/", 
        "cea":"./data/Dataset/HardTablesR2/gt/cea.csv",
        "cpa":"./data/Dataset/HardTablesR2/gt/cpa.csv",
        "cta": "./data/Dataset/HardTablesR2/gt/cta.csv"
    },
    "HardTableR3-2021": {
        "tables":"./data/Dataset/HardTablesR3/tables/", 
        "cea":"./data/Dataset/HardTablesR3/gt/cea.csv",
        "cpa":"./data/Dataset/HardTablesR3/gt/cpa.csv",
        "cta": "./data/Dataset/HardTablesR3/gt/cta.csv"
    }
}



id_dataset = "2T-2020"
tables_path, cea_target_path, cpa_target_path, cta_target_path = list(dataset[id_dataset].values())
id_dataset = f"{id_dataset}_TEST"
#buffer = utils.make_buffer(id_dataset, tables_path, cea_target_path, cpa_target_path, cta_target_path, kg_reference="wikidata", candidate_size=10)

In [18]:
import json
import os
import statistics
import sys

import pandas as pd
from dateutil.parser import parse
from pymongo import MongoClient
from tqdm import tqdm


DBNAME = os.environ["MONGO_DBNAME"]
MONGO_ENDPOINT, MONGO_ENDPOINT_PORT = os.environ["MONGO_ENDPOINT"].split(":")
MONGO_ENDPOINT_USERNAME = os.environ["MONGO_INITDB_ROOT_USERNAME"]
MONGO_ENDPOINT_PASSWORD = os.environ["MONGO_INITDB_ROOT_PASSWORD"]
client = MongoClient(MONGO_ENDPOINT, int(MONGO_ENDPOINT_PORT), username=MONGO_ENDPOINT_USERNAME, password=MONGO_ENDPOINT_PASSWORD)
cea_c = client[DBNAME].cea
#cea_c = client[DBNAME].ceaPrelinking
#cea_c = client[DBNAME].ceaInit
candidate_scored_c = client[DBNAME].candidateScored

def get_my_cea_annotation_scored(id_dataset):
    results = candidate_scored_c.find({"datasetName": id_dataset})
    total = candidate_scored_c.count_documents({"datasetName": id_dataset})
    out = {}
    for i, result in enumerate(tqdm(results, total=total)):
        id_table = result["tableName"]
        id_row = result["row"]
        for id_col, items in enumerate(result["candidates"]):
            key = f"{id_table} {id_row} {id_col}"
            out[key] = set()
            for item in items:
                #item = item[1]
                out[key].add(item["id"].lower())

        if i == 5000:
            break
            
    return out 


my_annotations = get_my_cea_annotation_scored(id_dataset)
a = 0
cea_gt_path = cea_target_path
gt_ann = utils.get_cea_ann(cea_gt_path)
missings = []
for key in gt_ann:
    found = False
    for ann in gt_ann[key]:
        if ann in my_annotations.get(key, {}):
            found = True
            a += 1
            break
    if found:
        missings.append([key, [_id.upper() for _id in gt_ann[key]]])

100%|██████████| 667243/667243 [00:06<00:00, 109178.81it/s]


In [ ]:
missings

In [19]:
len([m[0] for m in missings]), len(set([m[0] for m in missings]))

(14662, 14662)

In [2]:
buffer = utils.make_buffer(id_dataset, tables_path, cea_target_path, cpa_target_path, cta_target_path, kg_reference="wikidata", candidate_size=100)

  7%|▋         | 1627/22207 [00:16<03:13, 106.36it/s]/opt/conda/lib/python3.10/site-packages/dateutil/parser/_parser.py:1207: UnknownTimezoneWarning: tzname SOHO identified but not understood.  Pass `tzinfos` argument in order to correctly return a timezone-aware datetime.  In a future version, this will raise an exception.
  warnings.warn("tzname {tzname} identified but not understood.  "
 21%|██        | 4593/22207 [00:44<02:42, 108.16it/s]/opt/conda/lib/python3.10/site-packages/dateutil/parser/_parser.py:1207: UnknownTimezoneWarning: tzname D identified but not understood.  Pass `tzinfos` argument in order to correctly return a timezone-aware datetime.  In a future version, this will raise an exception.
  warnings.warn("tzname {tzname} identified but not understood.  "
 35%|███▌      | 7874/22207 [01:16<02:23, 99.61it/s] /opt/conda/lib/python3.10/site-packages/dateutil/parser/_parser.py:1207: UnknownTimezoneWarning: tzname NEAT identified but not understood.  Pass `tzinfos` argument

In [ ]:
buffer[0:2]

In [ ]:
json_data

In [4]:
headers = {
    'accept': 'application/json',
    'Content-Type': 'application/json',
}

params = (
    ('token', 'selbat_2023'),
    ('kg', 'wikidata')
)

json_data = buffer#[0:2]
response = requests.post('http://api:5000/dataset/createWithArray', headers=headers, params=params, json=json_data)
result = response.json()
result

{'status': 'Ok',
 'tables': [{'id': '644ea0729a2193c9a771746e',
   'datasetName': 'Round4_TEST',
   'tableName': '23PWC8OY'},
  {'id': '644ea0729a2193c9a771746f',
   'datasetName': 'Round4_TEST',
   'tableName': '7P5E37D1'},
  {'id': '644ea0729a2193c9a7717470',
   'datasetName': 'Round4_TEST',
   'tableName': 'AIGRYHCI'},
  {'id': '644ea0729a2193c9a7717471',
   'datasetName': 'Round4_TEST',
   'tableName': 'H45N92OH'},
  {'id': '644ea0729a2193c9a7717472',
   'datasetName': 'Round4_TEST',
   'tableName': 'I4KZA5RJ'},
  {'id': '644ea0729a2193c9a7717473',
   'datasetName': 'Round4_TEST',
   'tableName': 'Q65SA3QI'},
  {'id': '644ea0729a2193c9a7717474',
   'datasetName': 'Round4_TEST',
   'tableName': '67OI20BA'},
  {'id': '644ea0729a2193c9a7717475',
   'datasetName': 'Round4_TEST',
   'tableName': '67OI20BA'},
  {'id': '644ea0729a2193c9a7717476',
   'datasetName': 'Round4_TEST',
   'tableName': 'AZWMDC04'},
  {'id': '644ea0729a2193c9a7717477',
   'datasetName': 'Round4_TEST',
   'tableNam

In [5]:
response.text

'<!doctype html>\n<html lang=en>\n  <head>\n    <title>AttributeError: module \'math\' has no attribute \'round\'\n // Werkzeug Debugger</title>\n    <link rel="stylesheet" href="?__debugger__=yes&amp;cmd=resource&amp;f=style.css">\n    <link rel="shortcut icon"\n        href="?__debugger__=yes&amp;cmd=resource&amp;f=console.png">\n    <script src="?__debugger__=yes&amp;cmd=resource&amp;f=debugger.js"></script>\n    <script>\n      var CONSOLE_MODE = false,\n          EVALEX = true,\n          EVALEX_TRUSTED = false,\n          SECRET = "5DlOfGqe2MxrQAO8EBQj";\n    </script>\n  </head>\n  <body style="background-color: #fff">\n    <div class="debugger">\n<h1>AttributeError</h1>\n<div class="detail">\n  <p class="errormsg">AttributeError: module &#x27;math&#x27; has no attribute &#x27;round&#x27;\n</p>\n</div>\n<h2 class="traceback">Traceback <em>(most recent call last)</em></h2>\n<div class="traceback">\n  <h3></h3>\n  <ul><li><div class="frame" id="frame-139831497750560">\n  <h4>File 

In [ ]:
headers = {
    'accept': 'application/json',
    'Content-Type': 'application/json',
}

params = (
    ('token', 'selbat_2022'),
    ('kg', 'wikidata')
)


for id_dataset in dataset:
    tables_path, cea_target_path, cpa_target_path, cta_target_path = list(dataset[id_dataset].values())
    id_dataset = f"{id_dataset}_100_m1"
    buffer = utils.make_buffer(id_dataset, tables_path, cea_target_path, cpa_target_path, cta_target_path, kg_reference="wikidata", candidate_size=100)
    json_data = {
        'json': buffer
    }
    response = requests.post('http://api:5000/upload-table', headers=headers, params=params, json=json_data)
    result = response.json()

In [72]:
len(buffer)

22438

In [118]:
buffer[0:1]

[{'name': '0FD9838J',
  'dataset': 'HardTableR2-2021_10',
  'header': ['col0', 'col1'],
  'rows': [{'idRow': 1, 'data': ['BSR Rennsteig Oberhof', '150']},
   {'idRow': 2, 'data': ['Adelaide Hunt Club', '30']},
   {'idRow': 3, 'data': ['WSV Clausthal-Zellerfeld', '500']},
   {'idRow': 4, 'data': ['Simostranda IL', '500']}],
  'semanticAnnotations': {},
  'metadata': {'column': [{'idColumn': 0, 'tag': 'SUBJ'},
    {'idColumn': 1, 'tag': 'LIT', 'datatype': 'NUMBER'}]},
  'kgReference': 'wikidata',
  'candidateSize': 10}]

In [7]:
buffer = utils.make_buffer_for_missing_entities(id_dataset, f"{id_dataset}_iteration_2", tables_path, cea_target_path, cpa_target_path, 
                                       cta_target_path, candidate_size=100)

100%|██████████| 871/871 [00:00<00:00, 887139.09it/s]


In [12]:
from process.wrapper.mongodb_conn import get_collection

id_dataset = id_dataset
id_dataset = f"{id_dataset}_iteration_1"
cea_c = get_collection('cea')
results = cea_c.find({"idDataset": id_dataset})
total = cea_c.count_documents({"idDataset": id_dataset})

buffer = []
for result in tqdm(results, total=total):
    winning_candidates = result["winningCandidates"]
    for id_col, column in enumerate(winning_candidates):
        for winning_candidate in column:
            score = winning_candidate["score"]
            if len(column) > 1 and False:
                score = 0
            buffer.append({
                "idDataset": result["idDataset"],
                "tableName": result["tableName"],
                "col": id_col,
                "row": result["row"],
                "data": result["data"],
                "score": score
            })
            break        

0it [00:00, ?it/s]


In [13]:
buffer[0:2]

[]

In [27]:
sum([1 for b in buffer if b["score"] == 0])

0

In [78]:
buffer_sorted = sorted(buffer, key = lambda x: x["score"], reverse=True)
buffer_sorted[0:2]

[{'idDataset': 'Round4_iteration_1',
  'tableName': '1U6PF4FY',
  'col': 3,
  'row': 14,
  'data': ['zinovyev', '90', 'mikhaylovka urban okrug', 'russia'],
  'score': 62.093},
 {'idDataset': 'Round4_iteration_1',
  'tableName': '1U6PF4FY',
  'col': 3,
  'row': 6,
  'data': ['kurin', '56', 'mikhaylovka urban okrug', 'russia'],
  'score': 61.624}]

In [79]:
len(buffer_sorted)

20359

In [80]:
idx = int(len(buffer_sorted)*60/100)
idx

12215

In [7]:
tables_buffer = {}
for result in tqdm(results, total=total):
    id_table = result["tableName"]
    name = id_table
    if id_table not in tables_buffer:
        tables_buffer[id_table] = {
            "name": id_table, 
            "dataset": id_dataset,
            "rows": [],
            "semanticAnnotations": {"cta": [], "cpa": []},
            "metadata": {"column": {}},
            "kgReference": "wikidata",
            "candidateSize": candidate_size
        }
        datatype = targets_datatype[id_table]
        column = []
        for id_col, _ in enumerate(result["winningCandidates"]):
            if id_col in datatype["NE"]: 
                column.append({"idColumn": id_col, "tag": "NE" if id_col != datatype["SUBJ"] else "SUBJ"})
                if name in cta_buffer and cta_buffer[name].get(str(id_col)) is not None and len(cta_buffer[name].get(str(id_col))) > 0:
                    tables_buffer[id_table]["semanticAnnotations"]["cta"].append({
                                                                "idColumn": id_col, 
                                                                "types": cta_buffer[name][str(id_col)].split(" "),
                                                                "typesWeights": cta_weights[name][str(id_col)]
                                                                })
            else:
                column.append({"idColumn": id_col, "tag": "LIT", "datatype": datatype["LIT_DATATYPE"][str(id_col)]})
            if name in cpa_buffer and cpa_buffer[name].get(str(id_col)) is not None and len(cpa_buffer[name].get(str(id_col))) > 0:
                tables_buffer[id_table]["semanticAnnotations"]["cpa"].append({
                                                            "idColumn": id_col, 
                                                            "predicates": cpa_buffer[name][str(id_col)].split(" "),
                                                            "predicatesWeights": cpa_weights[name][str(id_col)]
                                                            })    

  0%|          | 0/12962 [00:00<?, ?it/s]


In [81]:
id_dataset_old = id_dataset.split("_iteration")[0]
#id_dataset_old = "2T-2020"
#id_dataset_old = id_dataset
targets_datatype = utils.get_tables_target(tables_path, cea_target_path, cpa_target_path, cta_target_path)
cta_weights, cta_buffer = utils.get_cta_types_table_to_column(id_dataset_old)
cpa_weights, cpa_buffer = utils.get_cpa_types_table_to_column(id_dataset_old)

 42%|████▏     | 9396/22207 [01:52<02:31, 84.55it/s] /opt/conda/lib/python3.10/site-packages/dateutil/parser/_parser.py:1207: UnknownTimezoneWarning: tzname SOHO identified but not understood.  Pass `tzinfos` argument in order to correctly return a timezone-aware datetime.  In a future version, this will raise an exception.
  warnings.warn("tzname {tzname} identified but not understood.  "
 80%|████████  | 17850/22207 [03:33<00:49, 87.92it/s]/opt/conda/lib/python3.10/site-packages/dateutil/parser/_parser.py:1207: UnknownTimezoneWarning: tzname NEAT identified but not understood.  Pass `tzinfos` argument in order to correctly return a timezone-aware datetime.  In a future version, this will raise an exception.
  warnings.warn("tzname {tzname} identified but not understood.  "
 88%|████████▊ | 19636/22207 [03:54<00:30, 83.30it/s]/opt/conda/lib/python3.10/site-packages/dateutil/parser/_parser.py:1207: UnknownTimezoneWarning: tzname NEAT identified but not understood.  Pass `tzinfos` argum

In [83]:
id_dataset_old, id_dataset

('Round4', 'Round4_iteration_1')

In [64]:
tables_buffer = {}
table_to_row = {}
for item in buffer_sorted[idx:]:
    table_name = item["tableName"]
    row = item["row"]
    data = item["data"]
    cell = data[item["col"]]
    if table_name not in tables_buffer:
        tables_buffer[table_name] = {}
    if row not in tables_buffer[table_name]:
        tables_buffer[table_name][row] = {}
    if table_name not in table_to_row:   
        table_to_row[table_name] = {}
    tables_buffer[table_name][row][cell] = True
    table_to_row[table_name][row] = item["data"] 

In [ ]:
tables_buffer = {}
table_to_row = {}
for item in buffer_sorted[idx:]:
    table_name = item["tableName"]
    row = item["row"]
    if table_name not in tables_buffer:
        tables_buffer[table_name] = set()
    if table_name not in table_to_row:   
        table_to_row[table_name] = {}
    tables_buffer[table_name].add(row)
    table_to_row[table_name][row] = item["data"] 

In [67]:
item

{'idDataset': 'HardTableR2-2021',
 'tableName': '2H409OPL',
 'col': 0,
 'row': 5,
 'data': ['!454',
  '445.5',
  '0.0903372',
  '192.1999876537',
  '36.963',
  '58464000.0'],
 'score': 1.5}

In [66]:
tables_buffer["3N7YTFJM"][1]

{'english': True}

In [71]:
tables_buffer = {}
table_to_row = {}
for item in buffer_sorted[idx:]:
    table_name = item["tableName"]
    row = item["row"]
    data = item["data"]
    cell = data[item["col"]]
    if table_name not in tables_buffer:
        tables_buffer[table_name] = {}
    if row not in tables_buffer[table_name]:
        tables_buffer[table_name][row] = {}
    if table_name not in table_to_row:   
        table_to_row[table_name] = {}
    tables_buffer[table_name][row][cell] = True
    table_to_row[table_name][row] = item["data"] 

        
new_table_buffer = {}
SIZE = 25
for id_table in tables_buffer:
    new_table_buffer[id_table] = []
    buffer = []
    for idx, row in enumerate(tables_buffer[id_table]):
        buffer.append(row)
        if len(buffer) >= SIZE:
            new_table_buffer[id_table].append(buffer)
            buffer = []
    if len(buffer) > 0:       
        new_table_buffer[id_table].append(buffer)      
new_table_buffer["3N7YTFJM"], tables_buffer["3N7YTFJM"][15]  

([[15, 18, 5, 19, 11, 16, 14, 1, 17, 10, 20, 9, 4, 6, 13, 8, 3, 2, 7, 12]],
 {'gothic': True})

In [79]:
idx

19

In [25]:
from process.wrapper.mongodb_conn import get_collection

for id_dataset in dataset:
    id_dataset_query = f"{id_dataset}_10"
    tables_path, cea_target_path, cpa_target_path, cta_target_path = list(dataset[id_dataset].values())
    targets_datatype = utils.get_tables_target(tables_path, cea_target_path, cpa_target_path, cta_target_path)
    cta_weights, cta_buffer = utils.get_cta_types_table_to_column(id_dataset_old)
    cpa_weights, cpa_buffer = utils.get_cpa_types_table_to_column(id_dataset_old)
    
    cea_c = get_collection('cea')
    results = cea_c.find({"idDataset": id_dataset_query})
    total = cea_c.count_documents({"idDataset": id_dataset_query})

    buffer = []
    for result in tqdm(results, total=total):
        winning_candidates = result["winningCandidates"]
        for id_col, column in enumerate(winning_candidates):
            for winning_candidate in column:
                score = winning_candidate["score"]
                if len(column) > 1 and False:
                    score = 0
                buffer.append({
                    "idDataset": result["idDataset"],
                    "tableName": result["tableName"],
                    "col": id_col,
                    "row": result["row"],
                    "data": result["data"],
                    "score": score
                })
    break        
    id_dataset_old = id_dataset


    tables_buffer = {}
    table_to_row = {}
    for item in buffer_sorted[idx:]:
        table_name = item["tableName"]
        row = item["row"]
        data = item["data"]
        cell = data[item["col"]]
        if table_name not in tables_buffer:
            tables_buffer[table_name] = {}
        if row not in tables_buffer[table_name]:
            tables_buffer[table_name][row] = {}
        if table_name not in table_to_row:   
            table_to_row[table_name] = {}
        tables_buffer[table_name][row][cell] = True
        table_to_row[table_name][row] = item["data"] 


    new_table_buffer = {}
    SIZE = 25
    for id_table in tables_buffer:
        new_table_buffer[id_table] = []
        buffer = []
        for row in tables_buffer[id_table]:
            buffer.append(row)
            if len(buffer) >= SIZE:
                new_table_buffer[id_table].append(buffer)
                buffer = []
        if len(buffer) > 0:       
            new_table_buffer[id_table].append(buffer)      

    buffer = []
   
    id_dataset_new = f"{id_dataset}_i1"
    for id_table in new_table_buffer:
        for rows in new_table_buffer[id_table]:
            table = {
                "name": id_table,
                "dataset": id_dataset_new,
                "rows": [],
                "semanticAnnotations": {"cta": [], "cpa": []},
                "metadata": {"column": {}},
                "kgReference": "wikidata",
                "candidateSize": 100
            }
            datatype = targets_datatype[id_table]
            name = id_table
            for row in rows:
                 table["rows"].append({"idRow": row, "data": table_to_row[id_table][row], "mentionsToReiterate": tables_buffer[id_table][row]})
            column = []
            for id_col, _ in enumerate(table["rows"][0]["data"]):
                if id_col in datatype["NE"]: 
                    column.append({"idColumn": id_col, "tag": "NE" if id_col != datatype["SUBJ"] else "SUBJ"})
                    if name in cta_buffer and cta_buffer[name].get(str(id_col)) is not None and len(cta_buffer[name].get(str(id_col))) > 0:
                        table["semanticAnnotations"]["cta"].append({
                                                                    "idColumn": id_col, 
                                                                    "types": cta_buffer[name][str(id_col)].split(" "),
                                                                    "typesWeights": cta_weights[name][str(id_col)]
                                                                    })
                elif id_col in datatype["LIT"]:
                    column.append({"idColumn": id_col, "tag": "LIT", "datatype": datatype["LIT_DATATYPE"][str(id_col)]})
                if name in cpa_buffer and cpa_buffer[name].get(str(id_col)) is not None and len(cpa_buffer[name].get(str(id_col))) > 0:
                    table["semanticAnnotations"]["cpa"].append({
                                                                "idColumn": id_col, 
                                                                "predicates": cpa_buffer[name][str(id_col)].split(" "),
                                                                "predicatesWeights": cpa_weights[name][str(id_col)]
                                                                })    
                table["metadata"]["column"] = column 
            buffer.append(table)
    break      

100%|██████████| 9089/9089 [00:01<00:00, 4839.57it/s]


In [42]:
from process.wrapper.mongodb_conn import get_collection

cea_c = get_collection('cea')
results = cea_c.find({"idDataset": id_dataset_query})
total = cea_c.count_documents({"idDataset": id_dataset_query})

buffer = []
for result in tqdm(results, total=total):
    winning_candidates = result["winningCandidates"]
    for id_col, column in enumerate(winning_candidates):
        for winning_candidate in column:
            score = winning_candidate["score"]
            if len(column) > 1 and False:
                score = 0
            buffer.append({
                "idDataset": result["idDataset"],
                "tableName": result["tableName"],
                "col": id_col,
                "row": result["row"],
                "data": result["data"],
                "score": score
            })
buffer_sorted = sorted(buffer, key = lambda x: x["score"], reverse=True)
idx = int(len(buffer_sorted)*60/100)            

100%|██████████| 58949/58949 [00:05<00:00, 10259.76it/s]


In [53]:
df = pd.read_csv("./data/iterations/HardTableR2-2021_10.csv")
rows = json.loads(df.to_json(orient="records"))
idx = int(len(rows)*40/100)

In [58]:
rows[idx:idx+2]

[{'datset': 'Round1_T2D',
  'key': '9834884_0_3871985887467090123 8 1',
  'gt': "['q1641836']",
  'id': 'q1641836',
  'score': 32.483,
  'outcome': 'OK'},
 {'datset': 'Round1_T2D',
  'key': '53822652_0_5767892317858575530 91 1',
  'gt': "['q1261193']",
  'id': 'q1261193',
  'score': 32.486,
  'outcome': 'OK'}]

In [ ]:
 'semanticAnnotations': {},
 'metadata': {'column': [{'idColumn': 0, 'tag': 'SUBJ'},
   {'idColumn': 1, 'tag': 'LIT', 'datatype': 'NUMBER'}]}

In [64]:
from process.wrapper.mongodb_conn import get_collection
from copy import deepcopy

headers = {
    'accept': 'application/json',
    'Content-Type': 'application/json',
    }

params = (
    ('token', 'selbat_2022'),
    ('kg', 'wikidata')
)


def get_rows_to_reteraite(rows, idx):
    rows_to_reiteraite = set()
    for item in rows[0:idx]:
        key = item["key"]
        rows_to_reiteraite.add(key)
    return rows_to_reiteraite    
            

row_collection = get_collection("row")

for id_dataset in dataset:
    cta_weights, cta_buffer = utils.get_cta_types_table_to_column(id_dataset)
    cpa_weights, cpa_buffer = utils.get_cpa_types_table_to_column(id_dataset)
    id_dataset_query = f"{id_dataset}_10"
    df = pd.read_csv(f"./data/iterations/{id_dataset_query}.csv")
    rows = json.loads(df.to_json(orient="records"))
    idx = int(len(rows)*40/100)
    rows_to_reiteraite = get_rows_to_reteraite(rows, idx)
    results = row_collection.find({"dataset": id_dataset_query})
    total = row_collection.count_documents({"dataset": id_dataset_query})
    buffer = []
    blocks = []
    table = {"name": None}
    for result in tqdm(results, total=total):
        name = result["name"]
        if name != table["name"]:
            if len(buffer) > 0:
                table["rows"] = deepcopy(buffer)
                blocks.append(table)
                buffer = []
            
        if len(buffer) == 0:
            metadata = result["metadata"]
            target = result["target"]
            for item in metadata["column"]:
                if item["tag"] == "LIT" and "datatype" not in item:
                    item["datatype"] =  target["LIT_DATATYPE"][str(item["idColumn"])]
            table = {
                "name": result["name"],
                "dataset": f"{id_dataset_query}0_i1_types",
                "header": result["header"],
                "rows": buffer,
                "semanticAnnotations": {"cpa": [], "cta": []},
                "metadata": result["metadata"],
                "kgReference": result["kgReference"],
                "candidateSize": 100
            }
            for id_col in cta_buffer[name]:
                if len(cta_buffer[name][id_col]) > 0:
                    table["semanticAnnotations"]["cta"].append({
                                                                "idColumn": id_col, 
                                                                "types": cta_buffer[name][id_col].split(" "),
                                                                "typesWeights": cta_weights[name][id_col]
                                                                })
            for id_col in cpa_buffer[name]:
                if len(cpa_buffer[name][id_col]) > 0:
                    table["semanticAnnotations"]["cpa"].append({
                                                                "idColumn": id_col, 
                                                                "predicates": cpa_buffer[name][id_col].split(" "),
                                                                "predicatesWeights": cpa_weights[name][id_col]
                                                                })        
        for item in result["rows"]:
            id_row = item["idRow"]
            put = False
            cell_to_size = {}
            new_data = []
            for id_col, cell in enumerate(item["data"]):
                key = f"{name} {id_row} {id_col}"
                if key in rows_to_reiteraite:
                    cell_to_size[cell] = 100
                    put = True
                else:
                    cell_to_size[cell] = 10
                cell_to_size[cell] = 100    
            item["cellToSize"] = cell_to_size
            put = True
            if put:
                buffer.append(item)
                
            if len(buffer) >= 25:
                table["rows"] = deepcopy(buffer)
                blocks.append(table)
                buffer = []
    if len(buffer) > 0:
        table["rows"] = deepcopy(buffer)
        blocks.append(table)
        buffer = []
    
  
    json_data = {
        'json': blocks
    }
    response = requests.post('http://api:5000/upload-table', headers=headers, params=params, json=json_data)
    result = response.json()

100%|██████████| 7207/7207 [00:02<00:00, 2618.01it/s]


In [ ]:
result

In [ ]:
table

In [20]:
temp

NameError: name 'temp' is not defined

In [95]:
key

'25404227_0_2240631045609013057 100 26310680_0_5150772059999313798'

In [58]:
blocks[-1]

{'name': 'QP0RM7QF',
 'dataset': 'HardTableR3-2021_10_i2',
 'header': ['col0', 'col1'],
 'rows': [{'idRow': 2,
   'data': ['Untitled', '5.0'],
   'cellToSize': {'Untitled': 100, '5.0': 10}},
  {'idRow': 3,
   'data': ['Catedral', '1.28'],
   'cellToSize': {'Catedral': 100, '1.28': 10}},
  {'idRow': 5,
   'data': ['A Roda', '0.8'],
   'cellToSize': {'A Roda': 100, '0.8': 10}},
  {'idRow': 6,
   'data': ['Colar', '7.6'],
   'cellToSize': {'Colar': 100, '7.6': 10}},
  {'idRow': 7,
   'data': ['Aranha', '4.5'],
   'cellToSize': {'Aranha': 100, '4.5': 10}},
  {'idRow': 8, 'data': ['OC', '0.3'], 'cellToSize': {'OC': 100, '0.3': 10}},
  {'idRow': 9,
   'data': ['Esfera', '2.0'],
   'cellToSize': {'Esfera': 100, '2.0': 10}},
  {'idRow': 11,
   'data': ['Guarany', '1.4'],
   'cellToSize': {'Guarany': 100, '1.4': 10}},
  {'idRow': 12, 'data': ['M', '7.0'], 'cellToSize': {'M': 100, '7.0': 10}},
  {'idRow': 16,
   'data': ['Goethe', '0.46'],
   'cellToSize': {'Goethe': 100, '0.46': 10}},
  {'idRow

In [94]:
buffer

[]

In [27]:
new_table_buffer["KUGCH9I3"][-1]

KeyError: 'KUGCH9I3'

In [35]:
output = buffer[0:2]
len(buffer)

0

In [ ]:
len(blocks), blocks[20]

In [34]:
output

NameError: name 'output' is not defined

In [102]:
table_to_row[id_table], id_table, tables_buffer[id_table]

({6: ['latin, fraktur type', 'german', '1600-01-01'],
  13: ['lycian', 'lycian', '-0499-01-01t00:00:00z'],
  8: ['shavian alphabet', 'esperanto', '1960-01-01'],
  3: ['tengwar', 'english', '1930-01-01'],
  1: ['avoiuli', 'english', '1990-01-01'],
  2: ['quikscript', 'english', '1966-01-01'],
  19: ['lao', 'lao', '1350-01-01'],
  7: ['night writing', 'french', '1815-01-01'],
  20: ['moon', 'armenian', '1845-01-01'],
  5: ['kurrent', 'german', '1911-01-01'],
  4: ['sütterlin', 'german', '1915-01-01'],
  12: ['lydian', 'lydian', '-0699-01-01t00:00:00z']},
 '3N7YTFJM',
 {1, 2, 3, 4, 5, 6, 7, 8, 12, 13, 19, 20})

In [62]:
datatype

{'SUBJ': 0, 'NE': [0, 1], 'LIT': [2], 'LIT_DATATYPE': {'2': 'DATETIME'}}

In [67]:
targets_datatype = utils.get_tables_target(tables_path, cea_target_path, cpa_target_path, cta_target_path)  
cta_weights, cta_buffer = utils.get_cta_types_table_to_column(id_dataset)
cpa_weights, cpa_buffer = utils.get_cpa_types_table_to_column(id_dataset)

100%|██████████| 5589/5589 [00:00<00:00, 145185.65it/s]


In [68]:
targets_datatype["3N7YTFJM"]

{'SUBJ': 0, 'NE': [0, 1], 'LIT': [2], 'LIT_DATATYPE': {'2': 'DATETIME'}}

In [51]:
buffer = blocks

In [54]:
headers = {
    'accept': 'application/json',
    'Content-Type': 'application/json',
}

params = (
    ('token', 'selbat_2022'),
    ('kg', 'wikidata')
)

json_data = {
    'json': buffer
}
response = requests.post('http://api:5000/upload-table', headers=headers, params=params, json=json_data)
result = response.json()
result

{'status': 'Ok',
 'tables': [{'id': '63a5a9522e5e8aac05788e21',
   'name': '22864497_0_8632623712684511496',
   'dataset': 'Round1_T2D_10_i2'},
  {'id': '63a5a9522e5e8aac05788e22',
   'name': '22864497_0_8632623712684511496',
   'dataset': 'Round1_T2D_10_i2'},
  {'id': '63a5a9522e5e8aac05788e23',
   'name': '22864497_0_8632623712684511496',
   'dataset': 'Round1_T2D_10_i2'},
  {'id': '63a5a9522e5e8aac05788e24',
   'name': '22864497_0_8632623712684511496',
   'dataset': 'Round1_T2D_10_i2'},
  {'id': '63a5a9522e5e8aac05788e25',
   'name': '22864497_0_8632623712684511496',
   'dataset': 'Round1_T2D_10_i2'},
  {'id': '63a5a9522e5e8aac05788e26',
   'name': '22864497_0_8632623712684511496',
   'dataset': 'Round1_T2D_10_i2'},
  {'id': '63a5a9522e5e8aac05788e27',
   'name': '22864497_0_8632623712684511496',
   'dataset': 'Round1_T2D_10_i2'},
  {'id': '63a5a9522e5e8aac05788e28',
   'name': '22864497_0_8632623712684511496',
   'dataset': 'Round1_T2D_10_i2'},
  {'id': '63a5a9522e5e8aac05788e29',


In [ ]:
buffer[0]

In [ ]:
[item for item in buffer if item["name"] == "1438042986423_95_20150728002306-00329-ip-10-236-191-2_805336391_10"]

In [23]:
import json
data = pd.read_csv("Norm di film table - Foglio1.csv")
output = []
for i, row in enumerate(data.values.tolist()):
    row[-1] = int(row[-1].replace(',', ''))
    row = [str(cell) for cell in row]
    output.append({"idRow": i+1, "data": row})

In [24]:
print(json.dumps(output))

[{"idRow": 1, "data": ["Jurassic World", "Colin Trevorrow", "2015", "Universal Pictures", "124", "1670400637"]}, {"idRow": 2, "data": ["Superman Returns", "Bryan Singer", "2006", "Warner Bros.", "154", "391081192"]}, {"idRow": 3, "data": ["Batman Begins", "Christopher Nolan", "2005", "Warner Bros.", "140", "371853783"]}, {"idRow": 4, "data": ["Avatar", "James Cameron", "2009", "20 Century Fox", "162", "2744336793"]}]


In [2]:
import pandas as pd
import os
from tqdm import tqdm
from tqdm import tqdm
import json
import utils.utils as utils


tables_path = "./data/BiodivTab_DBP_2022/val/tables/" # put here the path of tables dataset
cea_target_path = './data/BiodivTab_DBP_2022/val/gt/CEA_biodivtab_gt.csv'
cpa_target_path = None
cta_target_path = './data/BiodivTab_DBP_2022/val/gt/CTA_biodivtab_gt.csv'
id_dataset = "BioDiv_2021" # put here the dataset name your to gave

In [2]:
buffer = utils.make_buffer(id_dataset, tables_path, cea_target_path, cpa_target_path, cta_target_path, kg_reference="dbpedia", candidate_size=100)

100%|██████████| 6/6 [00:00<00:00, 142.40it/s]


In [10]:
df =pd.read_csv('data/BiodivTab_DBP_2022/val/tables/0dc3add04e344228bb140d5392399521.csv')
df.columns

Index(['Samplecode', 'Sample.Collector', 'Species', 'Year_Collected',
       'Exp_Site', 'Exp_vicinity', 'Exp_Plot', 'Exp_Plot_Position', 'TAG',
       'neighbour.tree_TAG', 'shrub.TAG', 'Leaf.fresh.weight',
       'Leaf.dry.weight', 'LA', 'SLA', 'LDMC', 'N', 'C', 'CN', 'K', 'Mg', 'Ca',
       'Al', 'Ca2', 'Fe', 'K2', 'Mg2', 'Mn', 'Na', 'P', 'S', 'Zn', 'Co', 'Cu',
       'Pb', 'Sr', 'Cr', 'Ni', 'stomata.density', 'average.length',
       'average.width', 'Phenolics', 'Tannin'],
      dtype='object')

In [ ]:
buffer[0:5]

In [3]:
for id_dataset in dataset:
    tables_path, cea_target_path, cpa_target_path, cta_target_path = list(dataset[id_dataset].values())
    buffer = utils.make_buffer(id_dataset, tables_path, cea_target_path, cpa_target_path, cta_target_path, kg_reference="wikidata", candidate_size=10)

    headers = {
        'accept': 'application/json',
        'Content-Type': 'application/json',
    }

    params = (
        ('token', 'challenge_2022'),
        ('kg', 'wikidata')
    )

    json_data = {
        'json': buffer[0:5]
    }
    response = requests.post('http://api:5000/upload-table', headers=headers, params=params, json=json_data)
    result = response.json()

NameError: name 'dataset' is not defined

In [7]:
buffer = utils.make_buffer(id_dataset, tables_path, cea_target_path, cpa_target_path, cta_target_path, kg_reference="wikidata", candidate_size=100)

100%|██████████| 36/36 [00:00<00:00, 128.99it/s]


In [ ]:
len(buffer), buffer[0:2]

In [ ]:
targets_datatype["9b73e5e8f3894884bddaf5ca6997d99c"]

In [ ]:
cea_gt = pd.read_csv(cea_target_path, sep=',', header=None)


In [ ]:
cea_gt[1].mean()<=cea_gt[2].mean()


In [ ]:
new_columns = list(cea_gt.columns)
new_columns[1], new_columns[2] = 2, 1

In [ ]:
cea_gt.reindex(new_columns)

In [5]:
buffer[0]['metadata']['column'] = []

In [6]:
buffer[0]

{'name': '4G0UEN8O',
 'dataset': 'HardTableR2-2021',
 'rows': [{'idRow': 1, 'data': ['tantalum-190', '117', 'tungsten-190']},
  {'idRow': 2, 'data': ['tantalum-189', '116', 'tungsten-189']},
  {'idRow': 3, 'data': ['tantalum-188', '115', 'tungsten-188']},
  {'idRow': 4, 'data': ['tantalum-187', '114', 'tungsten-187']},
  {'idRow': 5, 'data': ['tantalum-186m', '113', 'tungsten-186']},
  {'idRow': 6, 'data': ['tantalum-186', '113', 'tungsten-186']},
  {'idRow': 7, 'data': ['tantalum-185', '112', 'tungsten-185']},
  {'idRow': 8, 'data': ['tantalum-184', '111', 'tungsten-184']},
  {'idRow': 9, 'data': ['tantalum-183', '110', 'tungsten-183']},
  {'idRow': 10, 'data': ['tantalum-182', '109', 'tungsten-182']},
  {'idRow': 11, 'data': ['tantalum-182m2', '109', 'tantalum-182']},
  {'idRow': 12, 'data': ['tantalum-182m1', '109', 'tantalum-182']},
  {'idRow': 13, 'data': ['tantalum-173', '100', 'hafnium-173']},
  {'idRow': 14, 'data': ['tantalum-172', '99', 'hafnium-172']},
  {'idRow': 15, 'data'

In [3]:
import requests
headers = {
    'accept': 'application/json',
    'Content-Type': 'application/json',
}

params = (
    ('token', 'selbat_2022'),
    ('kg', 'wikidata')
)

json_data = {
    'json': buffer[0:5]
}
response = requests.post('http://api:5000/upload-table', headers=headers, params=params, json=json_data)
response.json()

{'status': 'Ok',
 'tables': [{'id': '6338800862ef4ca9bfe0968c',
   'name': '23ec62734eee41768969b817069b4011',
   'dataset': 'BioDiv_2021'},
  {'id': '6338800862ef4ca9bfe0968d',
   'name': '23ec62734eee41768969b817069b4011',
   'dataset': 'BioDiv_2021'},
  {'id': '6338800862ef4ca9bfe0968e',
   'name': '23ec62734eee41768969b817069b4011',
   'dataset': 'BioDiv_2021'},
  {'id': '6338800862ef4ca9bfe0968f',
   'name': '9b73e5e8f3894884bddaf5ca6997d99c',
   'dataset': 'BioDiv_2021'},
  {'id': '6338800862ef4ca9bfe09690',
   'name': '9b73e5e8f3894884bddaf5ca6997d99c',
   'dataset': 'BioDiv_2021'}]}

In [ ]:
len(buffer), buffer[0:2]

In [ ]:
cea_target_path

In [ ]:
buffer = {}
utils.get_cell_tables_data_to_annotations(cea_target_path, tables_path, buffer)

In [ ]:
len(buffer)

In [ ]:
buffer["rome"]

In [ ]:
DBNAME

In [ ]:
from pymongo import MongoClient
from tqdm import tqdm
import os
from pymongo import MongoClient
from process.lamapi_cache import LamAPICacheWrapper


LAMAPI_ENDPOINT = os.environ["LAMAPI_ENDPOINT"]
LAMAPI_ENDPOINT_PORT = int(os.environ["LAMAPI_ENDPOINT_PORT"])
LAMAPI_ENDPOINT_USERNAME = os.environ["LAMAPI_ENDPOINT_USERNAME"]
LAMAPI_ENDPOINT_PASSWORD = os.environ["LAMAPI_ENDPOINT_PASSWORD"]


def get_lamapi_connector(kg_reference):
    mongo_lamapi = MongoClient(LAMAPI_ENDPOINT, LAMAPI_ENDPOINT_PORT, 
                                username=LAMAPI_ENDPOINT_USERNAME, password=LAMAPI_ENDPOINT_PASSWORD)
    objects_dump = mongo_lamapi[kg_reference].objects
    literals_dump = mongo_lamapi[kg_reference].literals
    types_dump = mongo_lamapi[kg_reference].types
    documents_dump = mongo_lamapi[kg_reference].documents
    labels_dump = mongo_lamapi[kg_reference].labels
    lcw = LamAPICacheWrapper(objects_dump, literals_dump, types_dump, documents_dump, labels_dump)
    return lcw

DBNAME = os.environ["MONGO_DBNAME"]
MONGO_ENDPOINT = os.environ["MONGO_ENDPOINT"]
MONGO_ENDPOINT_PORT = int(os.environ["MONGO_ENDPOINT_PORT"])
MONGO_ENDPOINT_USERNAME = os.environ["MONGO_INITDB_ROOT_USERNAME"]
MONGO_ENDPOINT_PASSWORD = os.environ["MONGO_INITDB_ROOT_PASSWORD"]
client = MongoClient(MONGO_ENDPOINT, MONGO_ENDPOINT_PORT, username=MONGO_ENDPOINT_USERNAME, password=MONGO_ENDPOINT_PASSWORD)
candidate_memory_c = client[DBNAME].candidateMemory
lcw = get_lamapi_connector("dbpedia")

out = []
acc = []
cell_to_process = []
SIZE = 500
for cell in tqdm(buffer):
    for candidate in buffer[cell]:
        acc.append(candidate)
    cell_to_process.append(cell)
    if len(acc) >= SIZE:    
        labels = lcw.get_labels(acc)
        documents = lcw.get_documents(acc)
        acc = []
        for cell in cell_to_process:
            out_local = []
            candidates = buffer[cell]
            for candidate in candidates:
                for label in labels.get(candidate, {"labels": []})["labels"]:
                    out_local.append({
                        "id": candidate,
                        "name": label,
                        "primaryName": label,
                        "description": documents[candidate]["description"],
                        "types": documents[candidate]["types"]
                    })
            out.append({
                "cell": cell,
                "candidates": out_local,
                "kg": "dbpedia"
            })    
        cell_to_process = []
    #candidate_memory_c

In [ ]:
lcw.get_documents(["Rome", "House"])

In [ ]:
out[0:3]

In [ ]:
len(out)

In [ ]:
candidate_memory_c.insert_many(out)

In [ ]:
for item in tqdm(out):
    try:
        candidate_memory_c.insert_one(item)
    except:
        pass

In [ ]:
_, buffer = utils.get_cta_types_table_to_column(id_dataset)
with open("2T-cta-top-20.json", "w") as f:
    f.write(json.dumps(buffer, indent=4))

# STEP1 
Load dataset

In [ ]:
my_annotations = utils.get_my_cea_annotation("2T-valid")
new_my_cea = utils.get_my_cea_annotation(id_dataset)
for key in new_my_cea:
    my_annotations[key] = new_my_cea[key]
gt_files = pd.read_csv(cea_target_path)
a = 0
for row in tqdm(gt_files.itertuples(), total=len(gt_files)):
    id_table, id_row, id_col, gt = (row[i] for i in range(1, 5))
    key = f"{id_table} {id_row} {id_col}"
    gt_ann = [a[31:]   for a in gt.split(' ')]
    if my_annotations.get(key) in gt_ann:
        a += 1
p = (a/len(my_annotations)) if len(my_annotations) > 0 else 0
r = (a/len(gt_files))
f1 = (2*p*r)/(p+r) if len(my_annotations) > 0 else 0
p, r, f1

In [ ]:
utils.cea_eval(id_dataset, cea_target_path, [f"{id_dataset}-type1", f"{id_dataset}-type2"])

In [ ]:
'test/Abbiville'.rfind("/") if 'test/Abbiville'.rfind("/") != -1  else 'pluto'

In [ ]:
cea_gt = utils.get_cea_ann(cea_target_path)

In [ ]:
gt_files = pd.read_csv(cea_target_path)
target = {}
for row in tqdm(gt_files.itertuples(), total=len(gt_files)):
    id_table, id_row, id_col, gt = (row[i] for i in range(1, 5))
    key = f"{id_table} {id_row} {id_col}"
    id_table, id_row, id_col, gt = (row[i] for i in range(1, 5))
    key = f"{id_table} {id_row} {id_col}"
    entities = gt.split(' ')
    idx = entities[0].rfind('/')
    entities = [a[idx+1:].lower() if idx != -1 else a[31:] for a in entities]
    target[key] = entities

In [ ]:
target

In [ ]:
cea_gt

In [ ]:
cta_weights, cta_buffer = utils.get_cta_types_table_to_column(id_dataset)
cta_weights, cta_buffer

In [ ]:
targets_datatype = utils.get_tables_target(tables_path, cea_target_path, cpa_target_path, cta_target_path)  

<b>Load target datatatypes for each table</b>

In [ ]:
targets_datatype["GVEZJIC0"]

<b>Evaluation on datataset with Groundtruth</b>

In [ ]:
ancestor_file_path = "./data/DataSets/HardTablesR1/Valid/gt/cta_gt_ancestor.json"
descendent_file_path = "./data/DataSets/HardTablesR1/Valid/gt/cta_gt_descendent.json"
(utils.cea_eval(id_dataset, cea_target_path),
utils.cpa_eval(id_dataset, cpa_target_path)), 
utils.cta_eval(id_dataset, cta_target_path, )

In [ ]:
tables_to_cta = utils.get_cta_types_table_to_column(id_dataset)

with open("tables_to_cta.json", "w") as f:
    f.write(json.dumps(tables_to_cta, indent=4))

In [ ]:
utils.get_my_cea_annotation(id_dataset)

In [ ]:
df_cea = pd.DataFrame(my_ann.items())
cea_ann_df = df_cea[0].str.split(" ", expand=True)
cea_ann_df[3] = df_cea[1]
cea_ann_df.to_csv("HardTables-CPA-WD.csv", index=False, header=None)

In [3]:
table_name = "23ec62734eee41768969b817069b4011" # put here name of table to show 
df = pd.read_csv(f"{tables_path}/{table_name}.csv")
df

,animal_id,name,datetime,monthyear,date_of_birth,outcome_type,outcome_subtype,animal_type,Gender_identity,age_upon_outcome,breed,color
0,A771830,*Bradley,2018-12-27T13:25:00,2018-12-27T13:25:00,5/10/2017,Adoption,Foster,Nuuered Mals Dog,Neutered Male,1 year,Pit Bull Mix,White/Blue
1,A779576,*Rajah,2018-10-01T17:03:00,2018-10-01T17:03:00,4/18/2018,Adoption,NaN,Neutered Male Cat,Neutered Male,5 months,Domestic Shorthair Mix,Brown Tabby
2,A741715,*Pebbles,2017-01-11T18:17:00,2017-01-11T18:17:00,3/7/2016,Adoption,NaN,Spaed Femalp Ckt,Spayed Female,10 months,Domestic Shorthair Mix,Calico
3,A658751,Benji,2016-11-13T13:38:00,2016-11-13T13:38:00,7/14/2011,Return to Owner,NaN,Neutered Male Dog,Neutered Male,5 years,Border Terrier Mix,Tan
4,A721285,NaN,2016-02-24T14:42:00,2016-02-24T14:42:00,2/24/2014,Euthanasia,Suffering,Other,Unknown,2 years,Raccoon Mix,Black/Gray
...,...,...,...,...,...,...,...,...,...,...,...,...
94,A746964,NaN,2017-04-11T18:59:00,2017-04-11T18:59:00,3/11/2017,Transfer,Partner,Cat,Unknown,4 weeks,Domestic Shorthair Mix,Brown/Black
95,A781384,NaN,2018-10-05T18:00:00,2018-10-05T18:00:00,4/2/2018,Adoption,NaN,Spayed Female Cat,Spayed Female,6 months,Domestic Shorthair Mix,Black/White
96,A701956,*Davis,2015-05-07T09:00:00,2015-05-07T09:00:00,9/6/2014,Transfer,SCRP,Spayd Female zaf,Spayed Female,7 months,Domestic Shorthair Mix,Brown Tabby
97,A742133,Smooches,2017-01-20T18:01:00,2017-01-20T18:01:00,1/15/2016,Return to Owner,NaN,Neutered Male Cat,Neutered Male,1 year,Domestic Shorthair Mix,Black/White


In [ ]:
tables_buffer[0]

In [ ]:
targets_datatype["BA8EUB1Q"]

In [ ]:
DBNAME = os.environ["MONGO_DBNAME"]
MONGO_ENDPOINT = os.environ["MONGO_ENDPOINT"]
MONGO_ENDPOINT_PORT = int(os.environ["MONGO_ENDPOINT_PORT"])
MONGO_ENDPOINT_USERNAME = os.environ["MONGO_INITDB_ROOT_USERNAME"]
MONGO_ENDPOINT_PASSWORD = os.environ["MONGO_INITDB_ROOT_PASSWORD"]
client = MongoClient(MONGO_ENDPOINT, MONGO_ENDPOINT_PORT, username=MONGO_ENDPOINT_USERNAME, password=MONGO_ENDPOINT_PASSWORD)
cta_c = client[DBNAME].cta
results = cta_c.aggregate([{ "$match": { "idDataset": id_dataset} },
    {
        "$project": {
            "idDataset": 1,
            "tableName":1,
            "vectors": {
                "$objectToArray": "$winningCandidates"
            }
        },
    },
    { "$unwind": "$vectors" },
    {
        "$group": {
           "_id": {"col":"$vectors.k", "tableName":"$tableName"},
          "count": {
            "$mergeObjects": "$vectors.v"
          },
        },
      },
    ])
list(results), id_dataset

In [ ]:
tables_filter = [table for table in tables_buffer if table["name"] == "E3ZK744Q"]
tables_filter

In [ ]:
import json
tables = os.listdir(tables_path)
tables_buffer = []
cta_weights, cta_buffer = utils.get_cta_types_table_to_column(id_dataset)
cpa_weights, cpa_buffer = utils.get_cpa_types_table_to_column(id_dataset)


for table in tqdm(tables):
    name = table[:-4]
    if table.startswith('.'):
        continue
    id_row = 1
    i = 0
    f = 25
    df = pd.read_csv(f'{tables_path}/{table}')    
    json_data = json.loads(df.to_json(orient='split'))
    for _ in range(0, round(len(df)/25)+1):
        if len(json_data["data"][i:f]) == 0:
            continue
        table = {
            "name": name,
            "dataset": id_dataset,
            "rows": [],
            "semanticAnnotations": {"cta": [], "cpa": []},
            "metadata": {"column": {}},
            "kgReference": "wikidata",
            "candidateSize": 300
        }
    
    column = []
    datatype = targets_datatype[table["name"]]
    for id_col, _ in enumerate(json_data["data"][0]):
        if id_col in datatype["NE"]: 
            column.append({"idColumn": id_col, "tag": "NE" if id_col != datatype["SUBJ"] else "SUBJ"})
            if name in cta_buffer and cta_buffer[name].get(str(id_col)) is not None and len(cta_buffer[name].get(str(id_col))) > 0:
                table["semanticAnnotations"]["cta"].append({
                                                            "idColumn": id_col, 
                                                            "types": cta_buffer[name][str(id_col)].split(" "),
                                                            "typesWeights": cta_weights[name][str(id_col)]
                                                            })
        else:
            column.append({"idColumn": id_col, "tag": "LIT", "datatype": datatype["LIT_DATATYPE"][str(id_col)]})
        if name in cpa_buffer and cpa_buffer[name].get(str(id_col)) is not None and len(cpa_buffer[name].get(str(id_col))) > 0:
            table["semanticAnnotations"]["cpa"].append({
                                                        "idColumn": id_col, 
                                                        "predicates": cpa_buffer[name][str(id_col)].split(" "),
                                                        "predicatesWeights": cpa_weights[name][str(id_col)]
                                                        })    
    table["metadata"]["column"] = column        
    rows = json_data["data"][i:f]
    i = f
    f += 25        
    for row in rows:
        table["rows"].append({"idRow": id_row, "data": [str(cell) for cell in row]})
        id_row += 1
    tables_buffer.append(table)    

In [ ]:
targets_datatype["4G0UEN8O"]

In [ ]:
tables_buffer[0]

In [ ]:
print(json.dumps({"json":[tables_buffer[0]]}, indent=4))

# STEP4 

Run all cells in indexing.ipynb

# STEP5 

Start computation

In [ ]:
import json
with open("targets.json", "w") as f:
    f.write(json.dumps(targets_datatype, indent=4))

In [ ]:
import requests
headers = {
    'accept': 'application/json',
    'Content-Type': 'application/json',
}

params = (
    ('token', 'challenge_2022'),
    ('kg', 'wikidata')
)

json_data = {
    'json': tables_filter
}
response = requests.post('http://api:5000/upload-table', headers=headers, params=params, json=json_data)
response.json()

In [ ]:
targets_datatype = utils.get_tables_target2(tables_path, cea_target_path, cta_target_path)  

# STEP6 
Running with column types on low confidence or unlinked cells

In [ ]:
import os
import re
import statistics
import json
import pandas as pd
from dateutil.parser import parse
from pymongo import MongoClient
from tqdm import tqdm

DBNAME = os.environ["MONGO_DBNAME"]
MONGO_ENDPOINT = os.environ["MONGO_ENDPOINT"]
MONGO_ENDPOINT_PORT = int(os.environ["MONGO_ENDPOINT_PORT"])
MONGO_ENDPOINT_USERNAME = os.environ["MONGO_INITDB_ROOT_USERNAME"]
MONGO_ENDPOINT_PASSWORD = os.environ["MONGO_INITDB_ROOT_PASSWORD"]
client = MongoClient(MONGO_ENDPOINT, MONGO_ENDPOINT_PORT, username=MONGO_ENDPOINT_USERNAME, password=MONGO_ENDPOINT_PASSWORD)
cea_c = client[DBNAME].cea

id_dataset2 = "2T-valid"
cta_weights, cta_buffer = utils.get_cta_types_table_to_column(id_dataset2)
cpa_weights, cpa_buffer = utils.get_cpa_types_table_to_column(id_dataset2)


results = cea_c.find({"idDataset": id_dataset2})
total = cea_c.count_documents({"idDataset": id_dataset2})
out = {}
tables_buffer = {}
for result in tqdm(results, total=total):
    id_table = result["tableName"]
    name = id_table
    if id_table not in tables_buffer:
        tables_buffer[id_table] = {
            "name": id_table, 
            "dataset": id_dataset,
            "rows": [],
            "semanticAnnotations": {"cta": [], "cpa": []},
            "metadata": {"column": {}},
            "kgReference": "wikidata",
            "candidateSize": 500
        }
        datatype = targets_datatype[id_table]
        column = []
        for id_col, _ in enumerate(result["winningCandidates"]):
            if id_col in datatype["NE"]: 
                column.append({"idColumn": id_col, "tag": "NE" if id_col != datatype["SUBJ"] else "SUBJ"})
                if name in cta_buffer and cta_buffer[name].get(str(id_col)) is not None and len(cta_buffer[name].get(str(id_col))) > 0:
                    tables_buffer[id_table]["semanticAnnotations"]["cta"].append({
                                                                "idColumn": id_col, 
                                                                "types": cta_buffer[name][str(id_col)].split(" "),
                                                                "typesWeights": cta_weights[name][str(id_col)]
                                                                })
            else:
                column.append({"idColumn": id_col, "tag": "LIT", "datatype": datatype["LIT_DATATYPE"][str(id_col)]})
            if name in cpa_buffer and cpa_buffer[name].get(str(id_col)) is not None and len(cpa_buffer[name].get(str(id_col))) > 0:
                tables_buffer[id_table]["semanticAnnotations"]["cpa"].append({
                                                            "idColumn": id_col, 
                                                            "predicates": cpa_buffer[name][str(id_col)].split(" "),
                                                            "predicatesWeights": cpa_weights[name][str(id_col)]
                                                            })    
        tables_buffer[id_table]["metadata"]["column"] = column        
    id_row = result["row"]
    match = True
    for column in result["winningCandidates"]:
        if len(column) > 1 or (len(column) == 1 and column[0]["score"] < 1) or len(column) == 1:
            match = False
            break
    if not match: 
        tables_buffer[id_table]["rows"].append({"idRow": id_row, "data": result["data"]})

In [ ]:
buffer = []
for id_table in tables_buffer:
    if len(tables_buffer[id_table]["rows"]) > 0:
        buffer.append(tables_buffer[id_table])

In [ ]:
new_buffer = []
sample_size = 25
for table in tqdm(buffer):
    if len(table["rows"]) > sample_size:
        rows = []
        id_dataset = table["dataset"]
        id_table = table["name"]
        semantic_annotations = table["semanticAnnotations"]
        metadata = table["metadata"]
        for i, row in enumerate(table["rows"]):
            rows.append(row)
            if len(rows) >= sample_size and (len(table["rows"]) - i) > 25:
                new_buffer.append({
                    "name": id_table, 
                    "dataset": id_dataset,
                    "rows": rows,
                    "semanticAnnotations": semantic_annotations,
                    "metadata": metadata,
                    "kgReference": "wikidata",
                    "candidateSize": 500
                })
                rows = []
        if len(rows) > 0:
            new_buffer.append({
                    "name": id_table, 
                    "dataset": id_dataset,
                    "rows": rows,
                    "semanticAnnotations": semantic_annotations,
                    "metadata": metadata,
                    "kgReference": "wikidata",
                    "candidateSize": 500
            })
            rows = []
    else:
        new_buffer.append(table)

In [ ]:
len(buffer), len(new_buffer)

In [ ]:
new_buffer[0:10]

In [ ]:
[table for table in buffer if table["name"] == "R4CC666R"]

In [ ]:
freq = []
for table in new_buffer:
    freq.append((table["name"], len(table["rows"])))

In [ ]:
freq

In [ ]:
buffer[0]

In [ ]:
import requests
headers = {
    'accept': 'application/json',
    'Content-Type': 'application/json',
}

params = (
    ('token', 'challenge_2022'),
    ('kg', 'wikidata')
)

json_data = {
    'json': new_buffer
}
response = requests.post('http://api:5000/upload-table', headers=headers, params=params, json=json_data)
response.json()

In [ ]:
buffer[-1]

In [ ]:
cta_results = utils.get_cta_types_table_to_column(id_dataset)
len(cta_results)

In [ ]:
len(cta_results[0]), cta_results[0]

In [ ]:
import copy
new_cta_results = cta_results[0]
types_person = 0
temp = []
for id_table in new_cta_results:
    for id_col in new_cta_results[id_table]:
        if "Q5" in new_cta_results[id_table][id_col]:
            winning_types = sorted(new_cta_results[id_table][id_col].items(), key=lambda x: x[1], reverse=True)
            winning_type = winning_types[0][0]
            if new_cta_results[id_table][id_col]["Q5"] > 0.80 and winning_type != "Q5":
                temp.append([winning_types, id_table])                
                types_person += 1

In [ ]:
temp[0:3]

In [ ]:
winning_type

In [ ]:
types_person

In [ ]:
results = {
  "Tottenham": [
    {
      "id": "Tottenham",
      "label": "Tottenham",
      "description": "",
      "type": "Settlement Location Place PopulatedPlace",
      "ed_score": 1
    },
    {
      "id": "Tottenham,_Victoria",
      "label": "Tottenham",
      "description": "",
      "type": "Settlement Location Place PopulatedPlace",
      "ed_score": 1
    },
    {
      "id": "Tottenham_(1802_Indiaman)",
      "label": "Tottenham",
      "description": "",
      "type": "Ship MeanOfTransportation",
      "ed_score": 1
    },
    {
      "id": "Tottenham_railway_station",
      "label": "Tottenham",
      "description": "",
      "type": "Station ArchitecturalStructure Infrastructure",
      "ed_score": 1
    },
    {
      "id": "Tottenham,_New_South_Wales",
      "label": "Tottenham",
      "description": "",
      "type": "AdministrativeRegion Location Place PopulatedPlace Region",
      "ed_score": 1
    },
    {
      "id": "Tottenham_Hotspur_F.C.",
      "label": "FC Tottenham",
      "description": "",
      "type": "SoccerClub Agent Organisation SportsClub",
      "ed_score": 0.75
    },
    {
      "id": "Tottenham_Thunder",
      "label": "Tottenham Steam",
      "description": "",
      "type": "HockeyTeam Agent Organisation SportsTeam",
      "ed_score": 0.6
    },
    {
      "id": "Tottenham_Hotspur_F.C._Under-23s_and_Academy",
      "label": "Tottenham Hotspur",
      "description": "",
      "type": "SoccerClub Agent Organisation SportsClub",
      "ed_score": 0.53
    },
    {
      "id": "Tottenham_Hotspur_F.C._Women",
      "label": "Tottenham Hotspur LFC",
      "description": "",
      "type": "SoccerClub Agent Organisation SportsClub",
      "ed_score": 0.43
    },
    {
      "id": "Tottenham_Hotspur_(Superleague_Formula_team)",
      "label": "Tottenham Hotspur (Formula One)",
      "description": "",
      "type": "SoccerClub",
      "ed_score": 0.29
    },
    {
      "id": "2010–11_Tottenham_Hotspur_F.C._season",
      "label": "2010-11 Tottenham Hotspur season",
      "description": "",
      "type": "SoccerClubSeason SportsSeason SportsTeamSeason SoccerClub",
      "ed_score": 0.28
    },
    {
      "id": "2014–15_Tottenham_Hotspur_F.C._season",
      "label": "2014-15 Tottenham Hotspur season",
      "description": "",
      "type": "SoccerClubSeason SportsSeason SportsTeamSeason SoccerClub",
      "ed_score": 0.28
    },
    {
      "id": "2008–09_Tottenham_Hotspur_F.C._season",
      "label": "Tottenham Hotspur season 2008-09",
      "description": "",
      "type": "SoccerClubSeason SportsSeason SportsTeamSeason SoccerClub",
      "ed_score": 0.28
    },
    {
      "id": "SuperSport_United_F.C.",
      "label": "SuperSport Tottenham Hotspur Academy",
      "description": "",
      "type": "SoccerClub Agent Organisation SportsClub",
      "ed_score": 0.25
    },
    {
      "id": "1991–92_Tottenham_Hotspur_F.C._season",
      "label": "1991–92 Tottenham Hotspur F.C. season",
      "description": "",
      "type": "SoccerClubSeason SportsSeason SportsTeamSeason SoccerClub",
      "ed_score": 0.24
    },
    {
      "id": "1997–98_Tottenham_Hotspur_F.C._season",
      "label": "1997-98 Tottenham Hotspur F.C. season",
      "description": "",
      "type": "SoccerClubSeason SportsSeason SportsTeamSeason SoccerClub",
      "ed_score": 0.24
    },
    {
      "id": "2004–05_Tottenham_Hotspur_F.C._season",
      "label": "2004-05 Tottenham Hotspur F.C. season",
      "description": "",
      "type": "SoccerClubSeason SportsSeason SportsTeamSeason SoccerClub",
      "ed_score": 0.24
    },
    {
      "id": "2015–16_Tottenham_Hotspur_F.C._season",
      "label": "2015-16 Tottenham Hotspur F.C. season",
      "description": "",
      "type": "SoccerClubSeason SportsSeason SportsTeamSeason SoccerClub",
      "ed_score": 0.24
    },
    {
      "id": "2009–10_Tottenham_Hotspur_F.C._season",
      "label": "2009-10 Tottenham Hotspur F.C. season",
      "description": "",
      "type": "SoccerClubSeason SportsSeason SportsTeamSeason SoccerClub",
      "ed_score": 0.24
    },
    {
      "id": "1993–94_Tottenham_Hotspur_F.C._season",
      "label": "1993–94 Tottenham Hotspur F.C. season",
      "description": "",
      "type": "SoccerClubSeason SportsSeason SportsTeamSeason SoccerClub",
      "ed_score": 0.24
    },
    {
      "id": "2001–02_Tottenham_Hotspur_F.C._season",
      "label": "2001-02 Tottenham Hotspur F.C. season",
      "description": "",
      "type": "SoccerClubSeason SportsSeason SportsTeamSeason SoccerClub",
      "ed_score": 0.24
    },
    {
      "id": "2016–17_Tottenham_Hotspur_F.C._season",
      "label": "2016–17 Tottenham Hotspur F.C. season",
      "description": "",
      "type": "SoccerClubSeason SportsSeason SportsTeamSeason SoccerClub",
      "ed_score": 0.24
    },
    {
      "id": "1996–97_Tottenham_Hotspur_F.C._season",
      "label": "1996–97 Tottenham Hotspur F.C. season",
      "description": "",
      "type": "SoccerClubSeason SportsSeason SportsTeamSeason SoccerClub",
      "ed_score": 0.24
    },
    {
      "id": "2002–03_Tottenham_Hotspur_F.C._season",
      "label": "2002-03 Tottenham Hotspur F.C. season",
      "description": "",
      "type": "SoccerClubSeason SportsSeason SportsTeamSeason SoccerClub",
      "ed_score": 0.24
    },
    {
      "id": "2012–13_Tottenham_Hotspur_F.C._season",
      "label": "2012-13 Tottenham Hotspur F.C. season",
      "description": "",
      "type": "SoccerClubSeason SportsSeason SportsTeamSeason SoccerClub",
      "ed_score": 0.24
    },
    {
      "id": "2013–14_Tottenham_Hotspur_F.C._season",
      "label": "2013–14 Tottenham Hotspur F.C. season",
      "description": "",
      "type": "SoccerClubSeason SportsSeason SportsTeamSeason SoccerClub",
      "ed_score": 0.24
    },
    {
      "id": "1998–99_Tottenham_Hotspur_F.C._season",
      "label": "1998–99 Tottenham Hotspur F.C. season",
      "description": "",
      "type": "SoccerClubSeason SportsSeason SportsTeamSeason SoccerClub",
      "ed_score": 0.24
    },
    {
      "id": "2011–12_Tottenham_Hotspur_F.C._season",
      "label": "2011–12 Tottenham Hotspur F.C. season",
      "description": "",
      "type": "SoccerClubSeason SportsSeason SportsTeamSeason SoccerClub",
      "ed_score": 0.24
    },
    {
      "id": "2007–08_Tottenham_Hotspur_F.C._season",
      "label": "Tottenham Hotspur F.C. season 2007–08",
      "description": "",
      "type": "SoccerClubSeason SportsSeason SportsTeamSeason SoccerClub",
      "ed_score": 0.24
    },
    {
      "id": "1994–95_Tottenham_Hotspur_F.C._season",
      "label": "1994–95 Tottenham Hotspur F.C. season",
      "description": "",
      "type": "SoccerClubSeason SportsSeason SportsTeamSeason SoccerClub",
      "ed_score": 0.24
    },
    {
      "id": "2003–04_Tottenham_Hotspur_F.C._season",
      "label": "2003-04 Tottenham Hotspur F.C. season",
      "description": "",
      "type": "SoccerClubSeason SportsSeason SportsTeamSeason SoccerClub",
      "ed_score": 0.24
    },
    {
      "id": "1990–91_Tottenham_Hotspur_F.C._season",
      "label": "1990–91 Tottenham Hotspur F.C. season",
      "description": "",
      "type": "SoccerClubSeason SportsSeason SportsTeamSeason SoccerClub",
      "ed_score": 0.24
    },
    {
      "id": "1992–93_Tottenham_Hotspur_F.C._season",
      "label": "1992–93 Tottenham Hotspur F.C. season",
      "description": "",
      "type": "SoccerClubSeason SportsSeason SportsTeamSeason SoccerClub",
      "ed_score": 0.24
    },
    {
      "id": "1999–2000_Tottenham_Hotspur_F.C._season",
      "label": "1999–2000 Tottenham Hotspur F.C. season",
      "description": "",
      "type": "SoccerClubSeason SportsSeason SportsTeamSeason SoccerClub",
      "ed_score": 0.23
    },
    {
      "id": "West_Chester_Golden_Rams",
      "label": "West Chester Golden Rams football",
      "description": "",
      "type": "SportsTeam Agent Organisation SoccerClub",
      "ed_score": 0.18
    },
    {
      "id": "Seton_Hall_Pirates",
      "label": "Seton Hall Pirates",
      "description": "",
      "type": "SportsTeam Agent Organisation SoccerClub",
      "ed_score": 0.17
    },
    {
      "id": "San_Diego_Toreros",
      "label": "San Diego Toreros softball",
      "description": "",
      "type": "SportsTeam Agent Organisation SoccerClub",
      "ed_score": 0.15
    },
    {
      "id": "South_Carolina_Gamecocks",
      "label": "South Carolina Gamecocks track and field",
      "description": "",
      "type": "SportsTeam Agent Organisation SoccerClub",
      "ed_score": 0.12
    }
  ]
}

In [ ]:
buffer = results[list(results.keys())[0]]

In [ ]:
check = 'SportsTeam Agent Organisation SoccerClub'
types = "SoccerClub SportsTeam"
sum([t in check for t in types.split()])

In [ ]:
types = "SoccerClub SportsTeam"
#types = ' OR '.join(types.split())
sorted(buffer, key=lambda x: (sum([t in x["type"].split() for t in types.split()]) and x['ed_score'], -len(x['id'])), reverse=True)